<a href="https://colab.research.google.com/github/midhunjmes/midhun-26-02-24/blob/main/new.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [36]:
!pip install presidio_analyzer
!pip install faker

In [38]:
from presidio_analyzer import AnalyzerEngine
from faker import Faker

analyzer=AnalyzerEngine()
ent=analyzer.get_supported_entities()
for e in ent:
  print(e)

CRYPTO
URL
UK_NHS
IBAN_CODE
PHONE_NUMBER
IP_ADDRESS
UK_NINO
AU_TFN
LOCATION
US_DRIVER_LICENSE
AU_ACN
AU_ABN
AU_MEDICARE
DATE_TIME
SG_NRIC_FIN
CREDIT_CARD
IN_PASSPORT
IN_VEHICLE_REGISTRATION
EMAIL_ADDRESS
US_ITIN
NRP
US_PASSPORT
US_BANK_NUMBER
IN_VOTER
IN_PAN
US_SSN
MEDICAL_LICENSE
IN_AADHAAR
PERSON


In [49]:
import pandas as pd

def analyze_column(df):
  entity_columns = {}  # Initialize as a dictionary
  for col in df.columns:
    unique_values = df[col].dropna().astype(str).unique()[:25]
    entity_counts = {}

    for value in unique_values:
      results = analyzer.analyze(text=value, language="en")
      for result in results:
        entity_counts[result.entity_type] = entity_counts.get(result.entity_type, 0) + 1
    if entity_counts:
      predominant_entity = max(entity_counts, key=entity_counts.get)
      if predominant_entity not in entity_columns:
        entity_columns[predominant_entity] = []  # Assign a list to the entity key
      entity_columns[predominant_entity].append(col)
  return entity_columns  # Return the dictionary


def load_and_analyze(file_path):
  if file_path.endswith(".csv"):
    df = pd.read_csv(file_path, engine="python")
  elif file_path.endswith((".xls", ".xlsx")):
    df = pd.read_excel(file_path)
  else:
    raise ValueError("Unsupported file format. Provide a CSV or Excel file.")

  df_sample = df.drop_duplicates().head()
  classifeid_columns = analyze_column(df_sample)
  return classifeid_columns  # Return the dictionary

In [57]:
file="numerical.csv"
classifeid_columns=load_and_analyze(file)
df=pd.read_csv(file)
print(classifeid_columns)
# masked_df=anonymize_data(df,classifeid_columns)
# print(masked_df)
anonymize_csv(file,"masked.csv",classifeid_columns)

{'PERSON': ['Name'], 'URL': ['Email'], 'PHONE_NUMBER': ['Phone Number'], 'CREDIT_CARD': ['Credit Card'], 'LOCATION': ['Address']}


In [56]:
import pandas as pd
from faker import Faker

fake = Faker()
faker_mapping = {
    "PERSON": fake.name,
    "URL": fake.email,
    "PHONE_NUMBER": fake.phone_number,
    "CREDIT_CARD": fake.credit_card_number,
    "LOCATION": fake.address,
}

def anonymize_csv(input_file,output_file,classified_columns):
  df=pd.read_csv(input_file)
  column_to_entity={col:entity for entity,cols in classified_columns.items() for col in cols}
  for col,entity in column_to_entity.items():
    if col in df.columns and entity in faker_mapping:
      df[col]=df[col].apply(lambda _: faker_mapping[entity]())
  df.to_csv(output_file,index=False)

#

In [55]:
data = {
    "name": ["Alice", "Bob", "Charlie"],
    "city": ["New York", "Los Angeles", "Chicago"],
    "email": ["alice@example.com", "bob@example.com", "charlie@example.com"],
    "age": [25, 30, 35]  # Numerical column (ignored)
}

df = pd.DataFrame(data)
classified_columns = {  # This would come from Presidio analysis
    "PERSON": ["name"],
    "GPE": ["city"],
    "EMAIL": ["email"]
}

masked_df = anonymize_data(df, classified_columns)
print(masked_df)

KeyError: 'Name'